## 1. Importing data and data preprocessing

### 1.1 Import necesary libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import json
import random
import tensorflowjs as tfjs

### 1.2 Load and preprocess data

In [2]:
# Prepare training data
training_data = []

# Hold training
with open('hold_training.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        # Separate x and y coords
        training_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],0]) # 0 is hold
        
# Spill training
with open('spill_training.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        training_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],1]) # 1 is spill

# Shuffling
random.shuffle(training_data)

# Extract
train_coords = []
train_labels = []
minValX = float('inf')
maxValX = 0
minValY = float('inf')
maxValY = 0
for sample in training_data:
    train_coords.append(sample[0])
    train_labels.append(sample[1])
    minValX = min(minValX, min(sample[0][0]))
    maxValX = max(maxValX, max(sample[0][0]))
    minValY = min(minValY, min(sample[0][1]))
    maxValY = max(maxValY, max(sample[0][1]))
# Normalize
difValX = maxValX - minValX
difValY = maxValY - minValY
for i in range(len(train_coords)):
    for j in range(len(train_coords[i][0])):
        train_coords[i][0][j] = (train_coords[i][0][j] - minValX) / float(difValX)
        train_coords[i][1][j] = (train_coords[i][1][j] - minValY) / float(difValY)
        
        train_coords[i][0] = np.asarray(train_coords[i][0])
        train_coords[i][1] = np.asarray(train_coords[i][1])
    # Convert to np array
    train_coords[i] = np.asarray(train_coords[i])

# Convert
train_coords = np.asarray(train_coords)
train_labels = np.asarray(train_labels)
print(train_coords)
print(train_labels)

[[[0.35602094 0.34904014 0.34904014 ... 0.36736475 0.36736475 0.45287958]
  [0.58213256 0.60518732 0.60518732 ... 0.59654179 0.59654179 0.45244957]]

 [[0.37958115 0.37958115 0.37870855 ... 0.45811518 0.504363   0.504363  ]
  [0.42363112 0.42363112 0.42363112 ... 0.129683   0.22766571 0.22766571]]

 [[0.45375218 0.45375218 0.43368237 ... 0.57591623 0.57591623 0.57591623]
  [0.96253602 0.96253602 0.9740634  ... 0.39193084 0.39193084 0.39193084]]

 ...

 [[0.44153578 0.44153578 0.41884817 ... 0.30017452 0.41884817 0.41884817]
  [0.43804035 0.43804035 0.44956772 ... 0.10662824 0.27665706 0.27665706]]

 [[0.61082024 0.62739965 0.64485166 ... 0.59511344 0.58115183 0.58115183]
  [0.55907781 0.57348703 0.5907781  ... 0.42939481 0.42651297 0.42651297]]

 [[0.40139616 0.40139616 0.37521815 ... 0.35340314 0.43106457 0.43106457]
  [0.35158501 0.35158501 0.34005764 ... 0.24207493 0.30259366 0.30259366]]]
[1 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1
 0 1 1 1 1 0 0 0 1 

In [3]:
# Prepare testing data
testing_data = []

# Hold testing
with open('hold_testing.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        # Separate x and y coords
        testing_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],0]) # 0 is hold
        
# Spill testing
with open('spill_testing.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        testing_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],1]) # 1 is spill

# Shuffling
random.shuffle(testing_data)

# Extract
test_coords = []
test_labels = []
minValX = float('inf')
maxValX = 0
minValY = float('inf')
maxValY = 0
for sample in testing_data:
    test_coords.append(sample[0])
    test_labels.append(sample[1])
    minValX = min(minValX, min(sample[0][0]))
    maxValX = max(maxValX, max(sample[0][0]))
    minValY = min(minValY, min(sample[0][1]))
    maxValY = max(maxValY, max(sample[0][1]))
# Normalize
difValX = maxValX - minValX
difValY = maxValY - minValY
for i in range(len(test_coords)):
    for j in range(len(test_coords[i][0])):
        test_coords[i][0][j] = (test_coords[i][0][j] - minValX) / float(difValX)
        test_coords[i][1][j] = (test_coords[i][1][j] - minValY) / float(difValY)
        
        test_coords[i][0] = np.asarray(test_coords[i][0])
        test_coords[i][1] = np.asarray(test_coords[i][1])
    # Convert to np array
    test_coords[i] = np.asarray(test_coords[i])

# Convert
test_coords = np.asarray(test_coords)
test_labels = np.asarray(test_labels)
print(test_coords.shape)
print(len(test_labels))

(20, 2, 150)
20


## 2. CNN

### 2.1 Build model

In [4]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(2,150)), # Adjust num tracked here
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

### 2.2 Compile 

In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 2.3 Train

In [6]:
model.fit(train_coords, train_labels, epochs=40)

W0723 10:38:58.521854 4591044032 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
200/200 [==============================] - 0s 716us/sample - loss: 0.8008 - acc: 0.5550
Epoch 2/40
200/200 [==============================] - 0s 45us/sample - loss: 0.6822 - acc: 0.6100
Epoch 3/40
200/200 [==============================] - 0s 45us/sample - loss: 0.6568 - acc: 0.5950
Epoch 4/40
200/200 [==============================] - 0s 45us/sample - loss: 0.6306 - acc: 0.6250
Epoch 5/40
200/200 [==============================] - 0s 38us/sample - loss: 0.6179 - acc: 0.7200
Epoch 6/40
200/200 [==============================] - 0s 35us/sample - loss: 0.6007 - acc: 0.6650
Epoch 7/40
200/200 [==============================] - 0s 39us/sample - loss: 0.5729 - acc: 0.7500
Epoch 8/40
200/200 [==============================] - 0s 40us/sample - loss: 0.5549 - acc: 0.7600
Epoch 9/40
200/200 [==============================] - 0s 41us/sample - loss: 0.5484 - acc: 0.7750
Epoch 10/40
200/200 [==============================] - 0s 39us/sample - loss: 0.5244 - acc: 0.8400
Epoch 11/40
200/20

### 2.4 Results

In [7]:
test_loss, test_acc = model.evaluate(test_coords, test_labels)

20/20 [==============================] - 0s 868us/sample - loss: 0.2068 - acc: 1.0000


In [8]:
predictions = model.predict(test_coords)
print(predictions)

[[0.8055397 ]
 [0.99980265]
 [0.7796998 ]
 [0.33366197]
 [0.34389684]
 [0.8518066 ]
 [0.25477642]
 [0.9936719 ]
 [0.30859542]
 [0.2347152 ]
 [0.39567965]
 [0.12274435]
 [0.99572223]
 [0.9989158 ]
 [0.8741718 ]
 [0.9181582 ]
 [0.14112766]
 [0.99880207]
 [0.2445157 ]
 [0.3635958 ]]


### 2.5 Save model

In [9]:
# Python model
model.save('Model/model.h5')
# JS model
tfjs.converters.save_keras_model(model, './ModelJS')